In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "4"

In [2]:
import numpy as np
import pickle as pic

from sklearn.model_selection import train_test_split

In [3]:
import deep_learning

In [4]:
if True:
    with open("splitted_round_0.pic", "rb") as r:
        X_training, X_val, X_extraction = pic.load(r)
    with open("splitted_labels_1000000.pic", "rb") as r:
        y_training, y_val, y_extraction = pic.load(r)
else:
    with open("traces_round_0_adjusted_only.pic", "rb") as r:
        traces_round_0 = pic.load(r)
    with open("labels_1000000.pic", "rb") as r:
        rws_perms_labels, round_perms_labels, copy_perms_labels, rws_masks_labels, round_masks_labels = pic.load(r)

    X_total, y_total = deep_learning.prepare_data_dl(traces_round_0, round_perms_labels, copy_perms_labels, round_masks_labels, rws_perms_labels, rws_masks_labels)

    profile, test = train_test_split(np.arange(X_total.shape[0]), train_size=750_000, random_state=0)

    X_profiling, X_extraction = X_total[profile], X_total[test]
    train, val = train_test_split(np.arange(X_profiling.shape[0]), test_size=0.1, random_state=0)
    X_training, X_val = X_profiling[train], X_profiling[val]

    with open("splitted_round_0.pic", "wb") as w:
        pic.dump((X_training, X_val, X_extraction), w)

    if False:
        y_profiling = {}
        y_training = {}
        y_val = {}
        y_extraction = {}
        for label in y_total.keys():
            print(label, end="\r")
            y_profiling[label], y_extraction[label] = y_total[label][profile], y_total[label][test]
            y_training[label], y_val[label] = y_profiling[label][train], y_profiling[label][val]

        with open("splitted_labels_1000000.pic", "wb") as w:
            pic.dump((y_training, y_val, y_extraction), w)
    else:
        with open("splitted_labels_1000000.pic", "rb") as r:
            y_training, y_val, y_extraction = pic.load(r)


In [5]:
resnet = deep_learning.ResNetSCA(network="orig_round_0", epochs=1000, dataset_size=X_training.shape[0])
try:
    deep_learning.check_file_exists("./resnet_models/resnet_750000_orig_round_0_adjusted.keras")
    from tensorflow.keras.models import load_model
    resnet.model = load_model("./resnet_models/resnet_750000_orig_round_0_adjusted.keras")
except ValueError:
    pass

In [6]:
train_gen = deep_learning.DataGenerator(X_training, y_training)
val_gen = deep_learning.DataGenerator(X_val, y_val)

In [7]:
history = resnet.train_model_generator(train_gen, val_gen, "./resnet_models/resnet_750000_orig_round_0_adjusted.keras", patience=10)

Epoch 1/1000
10547/10547 [==============================] - 2078s 196ms/step - loss: 40.2961 - block_perm_0_output_loss: 1.3325 - mask_0_0_0_output_loss: 2.7834 - mask_0_0_1_output_loss: 2.7782 - mask_0_1_0_output_loss: 2.7835 - mask_0_1_1_output_loss: 2.7813 - mask_0_2_0_output_loss: 2.7859 - mask_0_2_1_output_loss: 2.7846 - mask_0_3_0_output_loss: 2.7872 - mask_0_3_1_output_loss: 2.7857 - mask_0_4_0_output_loss: 2.7839 - mask_0_4_1_output_loss: 2.7837 - mask_0_5_0_output_loss: 2.7811 - mask_0_5_1_output_loss: 2.7816 - mask_0_6_0_output_loss: 2.7806 - mask_0_6_1_output_loss: 2.7827 - block_perm_0_output_accuracy: 0.5197 - mask_0_0_0_output_accuracy: 0.0759 - mask_0_0_1_output_accuracy: 0.0773 - mask_0_1_0_output_accuracy: 0.0746 - mask_0_1_1_output_accuracy: 0.0764 - mask_0_2_0_output_accuracy: 0.0722 - mask_0_2_1_output_accuracy: 0.0738 - mask_0_3_0_output_accuracy: 0.0733 - mask_0_3_1_output_accuracy: 0.0742 - mask_0_4_0_output_accuracy: 0.0737 - mask_0_4_1_output_accuracy: 0.0753 -